# Pregunta 2

In [1]:
from util import go_to_project_root
go_to_project_root()

CWD cambiado a la raíz del proyecto: /home/epsilon/Documents/doctorado/S2/ENSTA630/tareas/HW2


In [41]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy.stats as stats
import statsmodels.distributions.empirical_distribution as ECDF

from source.display.hw import hw1 as uplot
from source.fit.error import get_standard_error
from source.models.ar import AutoRegressive
from source.models.ols import OLS
from source.fit.montecarlo import monte_carlo_mp
from source.data.transform import create_lagged_dataset


# from source.extention.pregunta2.source.models.autoregressive import AR
# from source.extention.pregunta2.source.models.Inferencia import Inferencia_OLS
# from source.extention.pregunta2.source.models.bootstrap import Bootstrap
# from source.extention.pregunta2.source.utils.criticos import valores_criticos
# from source.extention.pregunta2.source.utils.graficopdf import graficopdf
# from source.extention.pregunta2.source.utils.graficochow import graficochow

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
phi = [0.8, -0.2]
model = AutoRegressive(sigma=0.2, c=1, params_distribution=phi)
print(model)
model.get_autocovariance()

               AR(2) Model Summary                
Model Order              : 2
Intercept (c)            : 1
Error Std Dev (sigma)    : 0.2
Is Stationary            : True
Mu (Unc. Mean):          : 2.5
Sigma (Unc. Std.)        : 0.27
--------------------------------------------------
Coefficients (phi):
  phi_1    =  0.80
  phi_2    = -0.20


array([0.075, 0.05 ])

In [47]:
X = model.sample(n_samples=10)
y_prev, y_curr = create_lagged_dataset(X, lag=2)
ols_model = OLS()
ols_model.fit(y_prev, y_curr)
y_pred = ols_model.predict(y_prev)
get_standard_error(y_prev, y_pred, y_curr)

array([0.21257379, 0.21577446])

In [51]:
@monte_carlo_mp(n_iterations=500, n_jobs=-1)
def simulation_p2(model, n_samples_per_run, lags=1):
    X = model.sample(n_samples=n_samples_per_run+lags) # por el rezago
    # Calcular momentos
    moment_1 = np.mean(X)
    moment_2 = np.var(X)
    y_prev, y_curr = create_lagged_dataset(X, lag=lags)

    # OLS 
    ols_model = OLS()
    ols_model.fit(y_prev, y_curr)
    y_pred = ols_model.predict(y_prev)
    currsigma = get_standard_error(y_prev, y_pred, y_curr)

    return moment_1, moment_2, currsigma

In [52]:
simulation_p2(model, n_samples_per_run=10, lags=2)

Iniciando simulación paralela con 500 iteraciones en 8 procesos...


Simulación Monte Carlo:   0%|          | 0/500 [00:00<?, ?it/s]


PicklingError: Can't pickle <function simulation_p2 at 0x7a3b0921e340>: it's not the same object as __main__.simulation_p2

In [ ]:
np.random.seed(42)
# PREGUNTA 2.A: SIMULACIONES
R = np.array([[1,0,0], [0,1,4]]) # Restricciones
H0 = np.array([1,0]) # Valores bajo H0

simulaciones = []
inferencias = []
medias_iteraciones = []
varianza_iteraciones = []
for i in range(1000):
  muestra_actual = model.sample(n_samples=100)
  simulaciones.append(muestra_actual)
  muestra_actual = simulaciones[i]
  inferencia = Inferencia_OLS(muestra=muestra_actual, rezagos=2)
  inferencias.append(inferencia)
  medias_iteraciones.append(inferencia.get_momento1())
  varianza_iteraciones.append(inferencia.get_momento2())


# Para obtener el momento 1 estimado con Monte Carlo
momento1_estimado = np.mean(medias_iteraciones)
# Obtenemos la varianza estimada
momento2_estimado = np.mean(varianza_iteraciones)